In [ ]:
! git clone https://github.com/intel/e2eAIOK.git /tmp/e2eAIOK

In [ ]:
! apt-get update -y &&  DEBIAN_FRONTEND=noninteractive apt-get install -y python3 python3-pip python-is-python3 graphviz

In [ ]:
! DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre

In [ ]:
! cd /tmp/e2eAIOK/RecDP/ && python setup.py sdist && pip install dist/pyrecdp-*.tar.gz

In [ ]:
import os
from pyrecdp.primitives.llmutils import language_identify, language_identify_spark
from pyrecdp.primitives.llmutils.utils import get_target_file_list
from huggingface_hub import hf_hub_download

In [4]:
fasttext_model_dir = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
data_dir = "/tmp/e2eAIOK/RecDP/tests/data/llm_data"
data_files = get_target_file_list(data_dir, "jsonl", "file://")

In [ ]:
language_identify_output_dir = os.path.join(data_dir, "language_identify")
language_identify(data_dir, data_files, fasttext_model_dir, 'text', 'lang', language_identify_output_dir, "file://")

In [9]:
from pyrecdp.core import SparkDataProcessor
data_dir = "/tmp/e2eAIOK/RecDP/tests/data/llm_data"
data_file = "/tmp/e2eAIOK/RecDP/tests/data/llm_data/PILE/NIH_sample.jsonl"
save_path = os.path.join(data_dir, "language_identify/lid_df")
rdp = SparkDataProcessor()
spark=rdp.spark
spark_df = spark.read.json(data_file)
print("input is ")
spark_df.show()
lid_df = language_identify_spark(spark_df, fasttext_model_dir, 'text', 'lang', save_path)
print("output is")
lid_df.show()

Will assign 1 cores and 10386 M memory for spark
per core memory size is 10.143 GB and shuffle_disk maximum capacity is 8589934592.000 GB
input is 
+--------+--------------------+
|    meta|                text|
+--------+--------------------+
|{100065}|The National Dome...|
|{100066}|The Office of Pla...|
|{100067}|Improving outcome...|
|{100068}|This project is i...|
|{100069}|The CCDF Policies...|
|{100070}|The overall purpo...|
|{100071}|This contract wil...|
|{100072}|The purpose of th...|
|{100073}|The purpose of th...|
|{100074}|Intimate partner ...|
|{100075}|ACF's Office of R...|
|{100076}|The Temporary Ass...|
|{100077}|Investing in Qual...|
|{100078}|Current developme...|
|{100079}|The proposed diss...|
|{100080}|As the US populat...|
|{100081}|Through employing...|
|{100082}|The proposed mixe...|
|{100083}|To better serve D...|
|{100084}|A long standing r...|
+--------+--------------------+
only showing top 20 rows

process data started ...
process data took 0.7508777949997